# Average Shot Distance

In [19]:
import pandas as pd
import numpy as np
import yaml
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import teams

In [ ]:
# Get shot chart for entire league
shotdf = shotchartdetail.ShotChartDetail(
                team_id=0,
                player_id=0,
                context_measure_simple='FGA',
            ).get_data_frames()[0]

teams_list = teams.get_teams()

In [14]:
shotdf['POINT_VALUE'] = np.where(shotdf['SHOT_ZONE_BASIC'].str.contains('3'), '3', '2')

# Create aggregated dataframe for average distances
# Masks
is_three = shotdf['POINT_VALUE'] == '3'
is_two = shotdf['POINT_VALUE'] == '2'
made = shotdf['SHOT_MADE_FLAG'] == 1
missed = shotdf['SHOT_MADE_FLAG'] == 0

# Helper for agg
def agg_dist(df):
    return df.groupby('PLAYER_ID').agg(
        FGA=('SHOT_DISTANCE', 'count'),
        ALL_AVG_DISTANCE=('SHOT_DISTANCE', 'mean'),
        _3PT_AVG_DISTANCE=('SHOT_DISTANCE', lambda x: x[is_three.loc[x.index]].mean()),
        _2PT_AVG_DISTANCE=('SHOT_DISTANCE', lambda x: x[is_two.loc[x.index]].mean())
    ).rename(columns={
        '_3PT_AVG_DISTANCE': '3PT_AVG_DISTANCE',
        '_2PT_AVG_DISTANCE': '2PT_AVG_DISTANCE'
    })

# Full stats
all_df = agg_dist(shotdf).add_prefix('ALL_')
made_df = agg_dist(shotdf[made]).add_prefix('MADE_')
miss_df = agg_dist(shotdf[missed]).add_prefix('MISS_')

# Combine into one df
out = (
    all_df.join(made_df, how='outer')
          .join(miss_df, how='outer')
          .reset_index()
)

# Add player names and teams
names = shotdf[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID']].drop_duplicates(subset=['PLAYER_ID'])
out = out.merge(names, on='PLAYER_ID', how='left')
out['TEAM_ABBREVIATION'] = out['TEAM_ID'].map({team['id']: team['abbreviation'] for team in teams_list})

# Filter and re-order
out = out[out.ALL_FGA > 100].sort_values('MADE_2PT_AVG_DISTANCE', ascending=False)
out.to_csv('SHOOTING_DISTANCE_24-25.csv')

In [22]:
def fmt(val):
    return f"{val:.3f}" if pd.notnull(val) else ""

shot_distance_list = [{
    'id': str(row.PLAYER_ID),
    'name': str(row.PLAYER_NAME),
    'team': str(row.TEAM_ABBREVIATION),
    'fga': str(row.ALL_FGA),
    'all_avg_dist': fmt(row.ALL_ALL_AVG_DISTANCE),
    '3pt_avg_dist': fmt(row.ALL_3PT_AVG_DISTANCE),
    '2pt_avg_dist': fmt(row.ALL_2PT_AVG_DISTANCE),
    'made_fga': str(row.MADE_FGA),
    'made_all_avg_dist': fmt(row.MADE_ALL_AVG_DISTANCE),
    'made_3pt_avg_dist': fmt(row.MADE_3PT_AVG_DISTANCE),
    'made_2pt_avg_dist': fmt(row.MADE_2PT_AVG_DISTANCE),
    'miss_fga': str(row.MISS_FGA),
    'miss_all_avg_dist': fmt(row.MISS_ALL_AVG_DISTANCE),
    'miss_3pt_avg_dist': fmt(row.MISS_3PT_AVG_DISTANCE),
    'miss_2pt_avg_dist': fmt(row.MISS_2PT_AVG_DISTANCE),
} for _, row in out.iterrows()]


with open('shot_distance.yml', 'w') as stream:
  yaml.dump(shot_distance_list, stream)

In [16]:
out.columns

Index(['PLAYER_ID', 'ALL_FGA', 'ALL_ALL_AVG_DISTANCE', 'ALL_3PT_AVG_DISTANCE',
       'ALL_2PT_AVG_DISTANCE', 'MADE_FGA', 'MADE_ALL_AVG_DISTANCE',
       'MADE_3PT_AVG_DISTANCE', 'MADE_2PT_AVG_DISTANCE', 'MISS_FGA',
       'MISS_ALL_AVG_DISTANCE', 'MISS_3PT_AVG_DISTANCE',
       'MISS_2PT_AVG_DISTANCE', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION'],
      dtype='object')

In [70]:
active_players = players.get_players()

In [ ]:
test_id = '1630173'
active_players[id] == 

{'id': 1630173,
 'full_name': 'Precious Achiuwa',
 'first_name': 'Precious',
 'last_name': 'Achiuwa',
 'is_active': True}

In [ ]:
player_id = active_players[0]['id']

In [22]:
shotdf = shotchartdetail.ShotChartDetail(
                team_id=0,
                player_id=0,
                context_measure_simple='FGA',
            ).get_data_frames()[0]

In [17]:
shotdf.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [21]:
shotdf.SHOT_ZONE_BASIC.unique()

array(['Restricted Area', 'Right Corner 3', 'In The Paint (Non-RA)',
       'Mid-Range', 'Left Corner 3', 'Above the Break 3'], dtype=object)

In [27]:
unique_players = list(shotdf.PLAYER_ID.unique())

In [28]:
unique_players[0]

1642258

In [29]:
shotdf[shotdf.PLAYER_ID == unique_players[0]]

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022400001,7,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,11,43,...,Left Side Center(LC),24+ ft.,26,-168,205,1,0,20241112,BOS,ATL
9,Shot Chart Detail,0022400001,39,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,9,23,...,Center(C),Less Than 8 ft.,1,10,13,1,1,20241112,BOS,ATL
10,Shot Chart Detail,0022400001,46,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,50,...,Center(C),8-16 ft.,12,53,111,1,0,20241112,BOS,ATL
11,Shot Chart Detail,0022400001,48,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,47,...,Center(C),Less Than 8 ft.,1,-10,17,1,0,20241112,BOS,ATL
12,Shot Chart Detail,0022400001,50,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,46,...,Center(C),Less Than 8 ft.,1,-10,1,1,0,20241112,BOS,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214532,Shot Chart Detail,0022401202,580,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,4,7,20,...,Left Side(L),24+ ft.,24,-238,39,1,0,20241211,NYK,ATL
214533,Shot Chart Detail,0022401202,582,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,4,7,14,...,Center(C),Less Than 8 ft.,1,12,10,1,0,20241211,NYK,ATL
219191,Shot Chart Detail,0022401229,26,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,10,2,...,Left Side Center(LC),24+ ft.,26,-205,162,1,0,20241214,MIL,ATL
219238,Shot Chart Detail,0022401229,201,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,2,10,15,...,Center(C),Less Than 8 ft.,3,-33,17,1,0,20241214,MIL,ATL


In [30]:
shotdf[shotdf.SHOT_ZONE_BASIC.str.contains('3')]

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022400001,7,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,11,43,...,Left Side Center(LC),24+ ft.,26,-168,205,1,0,20241112,BOS,ATL
2,Shot Chart Detail,0022400001,12,1628401,Derrick White,1610612738,Boston Celtics,1,11,24,...,Left Side Center(LC),24+ ft.,28,-181,218,1,0,20241112,BOS,ATL
3,Shot Chart Detail,0022400001,21,1630552,Jalen Johnson,1610612737,Atlanta Hawks,1,10,50,...,Right Side Center(RC),24+ ft.,25,157,203,1,1,20241112,BOS,ATL
4,Shot Chart Detail,0022400001,23,1627759,Jaylen Brown,1610612738,Boston Celtics,1,10,35,...,Right Side Center(RC),24+ ft.,27,102,253,1,1,20241112,BOS,ATL
5,Shot Chart Detail,0022400001,31,1627759,Jaylen Brown,1610612738,Boston Celtics,1,10,6,...,Left Side(L),24+ ft.,24,-228,85,1,0,20241112,BOS,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219521,Shot Chart Detail,0022401230,703,1641717,Cason Wallace,1610612760,Oklahoma City Thunder,4,0,54,...,Right Side Center(RC),24+ ft.,25,188,168,1,1,20241214,OKC,HOU
219522,Shot Chart Detail,0022401230,705,1630578,Alperen Sengun,1610612745,Houston Rockets,4,0,47,...,Center(C),24+ ft.,25,-29,257,1,0,20241214,OKC,HOU
219523,Shot Chart Detail,0022401230,707,1627832,Fred VanVleet,1610612745,Houston Rockets,4,0,42,...,Left Side(L),24+ ft.,22,-228,-17,1,0,20241214,OKC,HOU
219524,Shot Chart Detail,0022401230,715,1629652,Luguentz Dort,1610612760,Oklahoma City Thunder,4,0,24,...,Right Side Center(RC),24+ ft.,24,186,167,1,1,20241214,OKC,HOU


In [32]:
shotdf['POINT_VALUE'] = np.where(shotdf['SHOT_ZONE_BASIC'].str.contains('3'), '3', '2')

In [ ]:
shotdf.groupby('POINT_VALUE')['SHOT_DISTANCE'].mean()

In [ ]:
# create mask columns
is_three = shotdf['POINT_VALUE'] == '3'
is_two = shotdf['POINT_VALUE'] == '2'

# group by player
out = shotdf.groupby('PLAYER_ID').agg(
    FGA=('SHOT_DISTANCE', 'count'),
    AVG_DISTANCE=('SHOT_DISTANCE', 'mean'),
    _3PT_AVG_DISTANCE=('SHOT_DISTANCE', lambda x: x[is_three].mean()),
    _2PT_AVG_DISTANCE=('SHOT_DISTANCE', lambda x: x[is_two].mean())
).reset_index()

# optional: rename columns to match spec exactly
out = out.rename(columns={
    '_3PT_AVG_DISTANCE': '3PT_AVG_DISTANCE',
    '_2PT_AVG_DISTANCE': '2PT_AVG_DISTANCE'
})

In [34]:
out

,PLAYER_ID,FGA,3PT_AVG_DISTANCE,2PT_AVG_DISTANCE
0,2544,1270,26.116751,7.027397
1,101108,583,25.149051,13.102804
2,200768,117,25.511628,12.129032
3,200782,7,25.333333,11.000000
4,201142,1124,25.018817,9.982713
...,...,...,...,...
561,1642461,34,23.764706,5.823529
562,1642484,33,25.350000,12.692308
563,1642502,2,23.000000,6.000000
564,1642505,30,24.190476,7.666667


In [46]:
# masks
is_three = shotdf['POINT_VALUE'] == '3'
is_two = shotdf['POINT_VALUE'] == '2'
made = shotdf['SHOT_MADE_FLAG'] == 1
missed = shotdf['SHOT_MADE_FLAG'] == 0

# helper for agg
def agg_dist(df):
    return df.groupby('PLAYER_ID').agg(
        FGA=('SHOT_DISTANCE', 'count'),
        ALL_AVG_DISTANCE=('SHOT_DISTANCE', 'mean'),
        _3PT_AVG_DISTANCE=('SHOT_DISTANCE', lambda x: x[is_three.loc[x.index]].mean()),
        _2PT_AVG_DISTANCE=('SHOT_DISTANCE', lambda x: x[is_two.loc[x.index]].mean())
    ).rename(columns={
        '_3PT_AVG_DISTANCE': '3PT_AVG_DISTANCE',
        '_2PT_AVG_DISTANCE': '2PT_AVG_DISTANCE'
    })

# full stats
all_df = agg_dist(shotdf).add_prefix('ALL_')
made_df = agg_dist(shotdf[made]).add_prefix('MADE_')
miss_df = agg_dist(shotdf[missed]).add_prefix('MISS_')

# combine
out = (
    all_df.join(made_df, how='outer')
          .join(miss_df, how='outer')
          .reset_index()
)

In [44]:
shotdf.to_csv('SHOT_CHART_DETAIL_24-25.csv')

In [45]:
out.to_csv('SHOOTING_DISTANCE_24-25.csv')

In [79]:
names = shotdf[['PLAYER_ID', 'PLAYER_NAME']].drop_duplicates()
out = out.merge(names, on='PLAYER_ID', how='left')

In [80]:
out[out.ALL_FGA > 100].sort_values('MADE_2PT_AVG_DISTANCE', ascending=False)

,PLAYER_ID,ALL_FGA,ALL_ALL_AVG_DISTANCE,ALL_3PT_AVG_DISTANCE,ALL_2PT_AVG_DISTANCE,MADE_FGA,MADE_ALL_AVG_DISTANCE,MADE_3PT_AVG_DISTANCE,MADE_2PT_AVG_DISTANCE,MISS_FGA,MISS_ALL_AVG_DISTANCE,MISS_3PT_AVG_DISTANCE,MISS_2PT_AVG_DISTANCE,PLAYER_NAME_x,PLAYER_NAME_y
422,1631260,424,23.485849,25.175000,13.984375,182.0,23.016484,24.928571,12.500000,242.0,23.838843,25.359223,15.138889,AJ Green,AJ Green
1,101108,583,20.727273,25.149051,13.102804,249.0,19.333333,25.121429,11.899083,334.0,21.766467,25.165939,14.352381,Chris Paul,Chris Paul
16,201942,1308,14.837156,25.104000,12.411153,624.0,13.219551,24.857143,11.409259,684.0,16.312865,25.228916,13.455598,DeMar DeRozan,DeMar DeRozan
54,203552,343,18.294461,24.752747,10.993789,164.0,17.701220,24.626506,10.604938,179.0,18.837989,24.858586,11.387500,Seth Curry,Seth Curry
43,203114,337,16.623145,25.340909,11.009756,160.0,14.843750,25.250000,10.383929,177.0,18.231638,25.392857,11.763441,Khris Middleton,Khris Middleton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,1642270,334,5.410180,25.122449,2.021053,180.0,3.316667,25.071429,1.481928,154.0,7.857143,25.142857,2.773109,Donovan Clingan,Donovan Clingan
13,201599,140,1.921429,NaN,1.921429,91.0,1.439560,NaN,1.439560,49.0,2.816327,NaN,2.816327,DeAndre Jordan,DeAndre Jordan
247,1630171,267,14.280899,23.768212,1.931034,124.0,11.419355,23.589286,1.397059,143.0,16.762238,23.873684,2.687500,Isaac Okoro,Isaac Okoro
50,203497,510,1.931373,NaN,1.931373,341.0,1.082111,NaN,1.082111,169.0,3.644970,NaN,3.644970,Rudy Gobert,Rudy Gobert


In [74]:
shotdf[shotdf.PLAYER_ID == 1642530]

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,POINT_VALUE
16259,Shot Chart Detail,0022400092,724,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,1,18,...,24+ ft.,26,-159,211,1,0,20241026,MEM,ORL,3
16262,Shot Chart Detail,0022400092,734,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,0,43,...,24+ ft.,26,-98,244,1,0,20241026,MEM,ORL,3
21944,Shot Chart Detail,0022400124,741,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,0,31,...,24+ ft.,25,29,254,1,0,20241030,MEM,BKN,3
22652,Shot Chart Detail,0022400128,697,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,1,21,...,24+ ft.,26,-170,205,1,0,20241031,MEM,MIL,3
30867,Shot Chart Detail,0022400174,662,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,1,40,...,8-16 ft.,13,-7,136,1,0,20241106,MEM,LAL,2
33851,Shot Chart Detail,0022400191,744,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,1,55,...,24+ ft.,26,100,247,1,1,20241108,MEM,WAS,3
33853,Shot Chart Detail,0022400191,747,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,1,44,...,24+ ft.,26,69,251,1,0,20241108,MEM,WAS,3
33857,Shot Chart Detail,0022400191,756,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,0,56,...,Less Than 8 ft.,2,18,18,1,0,20241108,MEM,WAS,2
37160,Shot Chart Detail,0022400210,623,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,6,36,...,24+ ft.,28,-172,222,1,0,20241110,POR,MEM,3
37176,Shot Chart Detail,0022400210,681,1642530,Yuki Kawamura,1610612763,Memphis Grizzlies,4,3,7,...,24+ ft.,25,-72,246,1,0,20241110,POR,MEM,3
